In [1]:
import numpy as np
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity
from skimage import img_as_ubyte
from natsort import natsorted
from skimage import exposure
from PIL import Image
from glob import glob
import os

def convert_to_8bit(image):
    """
    Converts a 16-bit image to an 8-bit image.
    """
    image = rescale_image(image, 16)
    return img_as_ubyte(image)

def rescale_image(image, bit_depth):
    """
    Rescales the image based on its bit depth.
    """
    return image / (2 ** bit_depth)

def calculate_metrics(rescaled_16bit, image_8bit):
    """
    Calculates MSE, PSNR, and SSIM between the original 16-bit and scaled 8-bit images.
    """
    mse = mean_squared_error(rescaled_16bit, image_8bit)
    psnr = peak_signal_noise_ratio(rescaled_16bit, image_8bit)
    ssim = structural_similarity(rescaled_16bit, image_8bit)

    return mse, psnr, ssim

# Get all 16-bit images paths
images_16bit  = natsorted(glob(os.path.join('dataset', 'Fluo-N2DL-HeLa', '01', '*.tif')))
images_16bit += natsorted(glob(os.path.join('dataset', 'Fluo-N2DL-HeLa', '02', '*.tif')))

# Metrics storage
mses, psnrs, ssims = [], [], []

# Process each image
for image_path in images_16bit:
    image_16bit = exposure.rescale_intensity(np.array(Image.open(image_path)), out_range=np.uint16)
    image_8bit = rescale_image(convert_to_8bit(image_16bit), 8)

    rescaled_16bit = rescale_image(image_16bit, 16)

    mse, psnr, ssim = calculate_metrics(rescaled_16bit, image_8bit)
    mses.append(mse)
    psnrs.append(psnr)
    ssims.append(ssim)

# Calculating mean and standard deviation for each metric
mse_mean, mse_std = np.mean(mses), np.std(mses)
psnr_mean, psnr_std = np.mean(psnrs), np.std(psnrs)
ssim_mean, ssim_std = np.mean(ssims), np.std(ssims)

print(f"MSE: Mean = {mse_mean}, Std = {mse_std}")
print(f"PSNR: Mean = {psnr_mean}, Std = {psnr_std}")
print(f"SSIM: Mean = {ssim_mean}, Std = {ssim_std}")


MSE: Mean = 1.3698370351271655e-06, Std = 5.59036273515446e-08
PSNR: Mean = 58.63686088060013, Std = 0.1747973327986176
SSIM: Mean = 0.9996486760005486, Std = 2.2954554874424678e-05
